# Project Desain Analisis Algoritma Sains Data Semester 3

### Kelompok 2

1. L0224012 - Trisha Garnis Wahyuningrun
2. L0224014 - Alvian Damar Budhi Hernowo
3. L0224037 - Talitha Sukma Mahardika

## Membuat Graph

In [57]:
import networkx as nx
import json
import random
import os

# Buat folder data jika belum ada
os.makedirs("data", exist_ok=True)

In [58]:
def generate_bem_graph(n_total, seed):
    random.seed(seed)
    n_a = int(n_total * 0.4)
    n_b = int(n_total * 0.35)
    n_c = n_total - n_a - n_b
    G_A = nx.barabasi_albert_graph(n=n_a, m=10, seed=seed)
    G_B = nx.barabasi_albert_graph(n=n_b, m=10, seed=seed+1)
    G_C = nx.barabasi_albert_graph(n=n_c, m=10, seed=seed+2)
    G_total = nx.disjoint_union_all([G_A, G_B, G_C])
    G_total.add_edge(0, n_a)             
    G_total.add_edge(n_a, n_a + n_b)
    G_total.add_edge(0, n_a + n_b)     
    node_data = {}
    nama = []
    file = open("data/nama.txt","a+")
    file.seek(0)
    for i in file.readlines() :
        nama.append(i)
    file.close()
    nnama = len(nama) - 1
    for node_id in G_total.nodes():
        if node_id < n_a:
            divisi = "Divisi Acara"
            jabatan = "Ketua Divisi" if node_id == 0 else "Staff"
        elif node_id < (n_a + n_b):
            divisi = "Divisi Humas"
            jabatan = "Ketua Divisi" if node_id == n_a else "Staff"
        else:
            divisi = "Divisi Logistik"
            jabatan = "Ketua Divisi" if node_id == (n_a + n_b) else "Staff"
        nama1 = nama.pop(random.randint(0,nnama)).strip()
        nama2 = nama.pop(random.randint(0,nnama-1)).strip()
        nnama -= 2
        node_data[str(node_id)] = {
            "nama": nama1 + " " + nama2,
            "divisi": divisi,
            "jabatan": jabatan,
            "ipk": round(random.uniform(2.5, 4.0), 2)
        }
    adj_list = nx.to_dict_of_lists(G_total)
    
    return adj_list, node_data

In [59]:
n = 375 
seed = 121437 
adj_list, node_data = generate_bem_graph(n, seed)      
output_content = {
    "project": "connected_components_social_graph",
    "description": f"Graf BEM {n} nodes dengan Data Atribut",
    "n_nodes": n,
    "graph_adj": adj_list,
    "node_data": node_data
}       
filename = f"data/social_graph_N{n}.json"
with open(filename, "w") as f:
    json.dump(output_content, f, indent=2)           
print(f"[OK] Berhasil membuat: {filename} (Lengkap dengan data mahasiswa)")

[OK] Berhasil membuat: data/social_graph_N375.json (Lengkap dengan data mahasiswa)


## DFS (Depth First Search)

In [60]:
def dfs(adj_list,start,goal,hasil) :
    for i in adj_list[start] :
        if i not in hasil :
            hasil[i] = start
            if i == goal :
                return True
            if dfs(adj_list,i,goal,hasil) :
                return True
    return False

## BFS (Breadth First Search)

In [61]:
def bfs(adj_list,start,goal,hasil) :
    qiu = []
    for i in adj_list[start] :
        if i not in hasil :
            hasil[i] = start
            qiu.append(i)
            if i == goal :
                return True
    while qiu :
        cabang = qiu.pop(0)
        for i in adj_list[cabang] :
            if i not in hasil :
                hasil[i] = cabang
                qiu.append(i)
            if i == goal :
                return True
    return False

## Hasil

In [68]:
hasil = {}
start = 171
goal = 200
if not (dfs(adj_list,start,goal,hasil)) :
    print("Tidak Ditemukan Jalur")
else :
    print(goal,end=" ")
    while True :
        print("-",hasil[goal],end=" ")
        goal = hasil[goal]
        if goal == start :
            break

200 - 189 - 183 - 197 - 202 - 187 - 190 - 188 - 198 - 184 - 194 - 181 - 178 - 179 - 182 - 176 - 173 - 175 - 172 - 177 - 180 - 174 - 160 - 171 

In [69]:
hasil = {}
start = 171
goal = 200
if not (bfs(adj_list,start,goal,hasil)) :
    print("Tidak Ditemukan Jalur")
else :
    print(goal,end=" ")
    while True :
        print("-",hasil[goal],end=" ")
        goal = hasil[goal]
        if goal == start :
            break

200 - 150 - 171 